<a href="https://colab.research.google.com/github/abdurrahmanshidiq/IBM-Capstone-Project/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw = requests.get(url).text

In [3]:
soup = BeautifulSoup(raw, 'html5lib')

In [4]:
len(soup.find('table'))

2

In [5]:
table = soup.find_all('table')[0].find_all('td')

post_code = []
borough = []
neigh = []

for i,row in enumerate(table):
  if table[i].span.text == 'Not assigned':
    pass
  else:
    post_code.append(table[i].p.text[:3])
    borough.append(table[i].span.text.split('(')[0])
    neigh.append(table[i].span.text.split('(')[1].replace('/',',').replace(')',''.strip(' ')))

In [6]:
# table_contents=[]
# table=soup.find('table')
# for row in table.findAll('td'):
#     cell = {}
#     if row.span.text=='Not assigned':
#         pass
#     else:
#         cell['PostalCode'] = row.p.text[:3]
#         cell['Borough'] = (row.span.text).split('(')[0]
#         cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
#         table_contents.append(cell)

# # print(table_contents)
# df=pd.DataFrame(table_contents)
# df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
#                                              'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
#                                              'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
#                                              'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [7]:
df_toronto = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto['PostalCode'] = post_code
df_toronto['Borough'] = borough
df_toronto['Neighborhood'] = neigh

df_toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

In [8]:
print(df_toronto[df_toronto['Borough']=='East YorkEast Toronto'].index)
print(df_toronto[df_toronto['Borough']=='MississaugaCanada Post Gateway Processing Centre'].index)
print(df_toronto[df_toronto['Borough']=='Downtown TorontoStn A PO Boxes25 The Esplanade'].index)
print(df_toronto[df_toronto['Borough']=='EtobicokeNorthwest'].index)
print(df_toronto[df_toronto['Borough']=='East TorontoBusiness reply mail Processing Centre969 Eastern'].index)

Int64Index([35], dtype='int64')
Int64Index([76], dtype='int64')
Int64Index([92], dtype='int64')
Int64Index([94], dtype='int64')
Int64Index([100], dtype='int64')


In [9]:
df_toronto.loc[35,'Borough'] = df_toronto.loc[35,'Borough'].replace('East YorkEast Toronto', 'East York')
df_toronto.loc[76,'Borough'] = df_toronto.loc[76,'Borough'].replace('MississaugaCanada Post Gateway Processing Centre', 'Mississauga')
df_toronto.loc[92,'Borough'] = df_toronto.loc[92,'Borough'].replace('Downtown TorontoStn A PO Boxes25 The Esplanade', 'Downtown Toronto')
df_toronto.loc[94,'Borough'] = df_toronto.loc[94,'Borough'].replace('EtobicokeNorthwest', 'Etobicoke')
df_toronto.loc[100,'Borough'] = df_toronto.loc[100,'Borough'].replace('East TorontoBusiness reply mail Processing Centre969 Eastern', 'East Toronto')

In [10]:
df_toronto

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [11]:
df_toronto.shape

(103, 3)

# Notebook 2

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [13]:
df_geo = pd.read_csv('/content/drive/My Drive/IBM DS Certificate/10. Capstone Project/2. K-Means Clustering/Geospatial_Coordinates.csv')
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']
df_geo

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [14]:
df_geo[df_geo['PostalCode']=='M5V']

,PostalCode,Latitude,Longitude
68,M5V,43.628947,-79.39442


In [15]:
df_merged = df_toronto.merge(df_geo, how='inner', on='PostalCode')
df_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


In [16]:
df_merged[df_merged['PostalCode']=='M5V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
87,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.39442
